In [1]:
import os
import logging
from dotenv import load_dotenv
import snowflake.connector

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Load environment variables
load_dotenv()

True

In [2]:
SNOWFLAKE_USER = os.getenv('SNOWFLAKE_USER')
SNOWFLAKE_PASSWORD = os.getenv('SNOWFLAKE_PASSWORD')
SNOWFLAKE_ACCOUNT = os.getenv('SNOWFLAKE_ACCOUNT')
SNOWFLAKE_WAREHOUSE = os.getenv('SNOWFLAKE_WAREHOUSE','FintechWH')
SNOWFLAKE_DATABASE = os.getenv('SNOWFLAKE_DATABASE','FintechDB')
SNOWFLAKE_SCHEMA = os.getenv('SNOWFLAKE_SCHEMA','FintechSchema')

In [3]:
print(SNOWFLAKE_PASSWORD)

Olamideade10%%


In [4]:

def get_snowflake_connection():
    """
    Establishes a connection to Snowflake using credentials from .env
    """
    try:
        conn = snowflake.connector.connect(
            user=SNOWFLAKE_USER,
            password=os.getenv('SNOWFLAKE_PASSWORD'),
            account=SNOWFLAKE_ACCOUNT,
            warehouse=SNOWFLAKE_WAREHOUSE,
            database=SNOWFLAKE_DATABASE,
            schema=SNOWFLAKE_SCHEMA
        )
        logging.info("Successfully connected to Snowflake")
        return conn
    except Exception as e:
        logging.error(f"Failed to connect to Snowflake: {str(e)}")
        raise


In [5]:


def upload_to_stage(file_path, stage_name='@~'):
    """
    Uploads a file to a Snowflake stage.
    
    Args:
        file_path (str): Path to the local file
        stage_name (str): Name of the stage (default is user stage @~)
    """
    conn = get_snowflake_connection()
    cursor = conn.cursor()
    
    try:
        # Snowflake PUT command expects forward slashes even on Windows for the file protocol
        # or properly escaped backslashes. Using forward slashes is safer.
        abs_path = os.path.abspath(file_path).replace('\\', '/')
        
        logging.info(f"Uploading {abs_path} to stage {stage_name}...")
        
        # PUT command to upload file
        # auto_compress=True is default, which is usually good for staging
        put_command = f"PUT file:///{abs_path} {stage_name} AUTO_COMPRESS=TRUE"
        
        cursor.execute(put_command)
        
        # Fetch results to verify
        results = cursor.fetchall()
        for source, target, source_size, target_size, status, message, *others in results:
            logging.info(f"Upload status for {source}: {status}")
            
    except Exception as e:
        logging.error(f"Error uploading file: {str(e)}")
    finally:
        cursor.close()
        conn.close()

if __name__ == "__main__":
    # Example usage with transactions.csv
    # Assuming the script is run from the project root
    dataset_names = ['customers.csv','transactions.csv','loans.csv','support_tickets.csv']  
    for names in dataset_names:
        csv_file = os.path.join('Datasets', names)
    
        if os.path.exists(csv_file):
            upload_to_stage(csv_file,"@RAW_DATA")
        else:
            logging.error(f"File not found: {csv_file}")

2025-12-04 10:30:49 - INFO - Snowflake Connector for Python Version: 4.1.1, Python Version: 3.10.11, Platform: Windows-10-10.0.22621-SP0
2025-12-04 10:30:49 - INFO - Connecting to GLOBAL Snowflake domain
2025-12-04 10:30:50 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-12-04 10:31:16 - INFO - Successfully connected to Snowflake
2025-12-04 10:31:16 - INFO - Uploading c:/Users/DELL/Documents/Fintech_Project/Datasets/customers.csv to stage @RAW_DATA...
2025-12-04 10:31:30 - WARNING - Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NameResolutionError("<snowflake.connector.vendored.urllib3.connection.HTTPSConnection object at 0x0000018BBD7D4C70>: Failed to resolve 'sfc-va4-ds1-27-customer-stage.s3.us-east-1.amazonaws.com' ([Errno 11001] getaddrinfo failed)")': /4eu91000-s/stages/daa5edc9-148a-46a8-9399-ccf1df7f2a8f/customers.csv.gz
2025-12-04 10:31:43 - WARNING - HEAD with url b'https://sfc-va4-ds1-